In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import json
import os
import time
import glob
import numpy as np
from bagpy import bagreader
import rosbag
import pandas as pd
import wandb

In [29]:
from screwing_dataset import ScrewingDataset
from screwing_model import ScrewingModel
from screwing_model_seq import ScrewingModelSeq


In [30]:
from torch.utils.data import DataLoader, ConcatDataset
import torch
import torch.nn as nn
import torch.optim as optim

In [31]:
from training import batched_pos_err, batched_ori_err, weighted_MSE_loss


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [74]:

batch_size = 1 # Powers of two
window_size = 30

input_dim = 19
hidden_dim = 10
num_layers = 3
output_dim = 5

#TODO change arbitrary weight
ori_rel_weight = 2

num_eps = 2

num_epochs = 200
learning_rate = 0.003

base_dset_dir = os.path.expanduser('~/datasets/screwing/')
# xprmnt_dir = time.strftime("/2022-03-10_23-17-39")
# xprmnt_dir = time.strftime("2022-03-11_17-07-13/")
xprmnt_dir = time.strftime("2022-04-04_14-40-37/")
# xprmnt_dir = time.strftime("2022-04-14_15-49-01/")


log_interval = 1 

train_ratio = .75


model_save_dir = '../../../models/'
model_name = 'model_2022-03-27_17-18-31.pt'

In [75]:
bag_path_names = base_dset_dir + xprmnt_dir + '*.bag' 

bag_path_list = glob.glob(bag_path_names)
total_num_eps = len(bag_path_list)
# wandb.config.update({'total_dset_eps_num': num_eps})

num_workers = 8


dset_list = []
# for i in range(num_eps): # for testing a small number of data
# for i in range(total_num_eps):
i = 0
id_str = str(i)
print(i)
bag_path_names = base_dset_dir + xprmnt_dir + id_str + '_*.bag' 

try:
    bag_path = glob.glob(bag_path_names)[0]
except:
    print('bag with index ' + str(i) + ' was not found! Skipping to next index')
    
bread = bagreader(bag_path)


0
[INFO]  Data folder /home/serialexperimentsleon/datasets/screwing/2022-04-04_14-40-37/0_2022-04-04-14-40-42 already exists. Not creating.


In [76]:
main_topic = '/panda/franka_state_controller_custom/franka_states'

In [77]:
topics = bread.topics
print(topics)
table = bread.topic_table
print(table)

dict_keys(['/panda/franka_state_controller_custom/franka_states', '/panda/franka_state_controller_custom/joint_states', '/screwing_primitives/pose_desired', '/tf', '/tf_static'])
                                              Topics  \
0  /panda/franka_state_controller_custom/franka_s...   
1  /panda/franka_state_controller_custom/joint_st...   
2                  /screwing_primitives/pose_desired   
3                                                /tf   
4                                         /tf_static   

                           Types  Message Count    Frequency  
0  franka_msgs/FrankaStateCustom            247    22.732616  
1         sensor_msgs/JointState            248    29.510959  
2      geometry_msgs/PoseStamped           2111   250.092660  
3             tf2_msgs/TFMessage            500    61.220884  
4             tf2_msgs/TFMessage              3  1630.122037  


In [78]:
topic_csv_dict = {}

In [79]:
topic = '/clock'
topic_csv_dict[topic] = bread.message_by_topic(topic)


No data on the topic:/clock


In [80]:
main_num_msgs = table[table['Topics'] == main_topic]['Message Count']
type(main_num_msgs)
print(main_num_msgs.item())
# main_num_msgs = table[table['Topics'] == main_topic]['Message Count'][0]


247
